In [ ]:
# https://github.com/olonok69/LLM_Notebooks/blob/main/Prompt_engineering/Mistral7_b_prompt_Engineering_Techniques.ipynb
# https://github.com/olonok69/LLM_Notebooks/blob/main/mistral/Mistral7b_instruct_qlora_CoT_Fine_Tune_v2.ipynb

# https://cloud.google.com/vertex-ai/docs/generative-ai/learn/overview

# pip install --upgrade google-cloud-aiplatform
# gcloud auth application-default login

In [13]:
import vertexai
from vertexai.language_models import TextGenerationModel
from vertexai.preview.generative_models import GenerativeModel, Part
import vertexai.preview.generative_models as generative_models

from dotenv import dotenv_values

config = dotenv_values(".env")

In [41]:
vertexai.init(project=config['PROJECT'], location=config['REGION'])
parameters = {
    "candidate_count": 1,
    "max_output_tokens": 1024,
    "temperature": 0.2,
    "top_p": 0.8,
    "top_k": 40
}
model = TextGenerationModel.from_pretrained("text-bison@001")


# Chain-of-Thought Prompting

In [35]:
response = model.predict(
    """Read the Instruction below and provide an answer.

### INSTRUCTION:
Question:
I had a car 20 years ago, and its cost was 100000$.
Car's anual rate depreciation is 5%. and no residual value
Using the Declining Balance method, what it is the value of the car now ?""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model: The answer is 100000 * 0.95 ^ 20 = 100000 * 0.00095 = 9.50$

The car's value depreciates by 5% every year. So, after 20 years, the car's value will be 100000 * 0.95 ^ 20 = 9.50$.


In [36]:
response = model.predict(
    """Read the Instruction below and provide an answer.

### INSTRUCTION:
Question:
I had a car 20 years ago, and its cost was 100000$.
Car's anual rate depreciation is 5%. and no residual value
Using the Declining Balance method, what it is the value of the car now ?
Let,s think step by step""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model: The answer is 100000 * 0.95 ^ 20 = 100000 * 0.0367 = 3670$.

The first step is to calculate the depreciation rate per year. This is done by dividing the annual rate of depreciation by 100. In this case, the annual rate of depreciation is 5%, so the depreciation rate per year is 5 / 100 = 0.05.

The next step is to calculate the depreciation for each year. This is done by multiplying the value of the car by the depreciation rate per year. In this case, the value of the car is $100,000, so the depreciation for each year is $100,000 * 0.05 = $5,000.

The final step is to calculate the value of the car after 20 years. This is done by subtracting the depreciation for each year from the value of the car. In this case, the value of the car after 20 years is $100,000 - $5,000 * 20 = $3670.


In [37]:
def generate():

  model = GenerativeModel("gemini-1.0-pro-001")
  responses = model.generate_content(
    """Read the Instruction below and provide an answer.

### INSTRUCTION:
Question:
I had a car 20 years ago, and its cost was 100000$.
Car\'s anual rate depreciation is 5%. and  it has no residual value.
Using the Declining Balance method, what it is the value of the car now ?
Let,s think step by step""",
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.1,
        "top_p": 1
    },
    safety_settings={
          generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    },
    stream=True,
  )
  
  for response in responses:
    print(response.text, end="")

generate()

**Step 1: Calculate the depreciation rate.**

Depreciation rate = Annual depreciation rate * Number of years
Depreciation rate = 5% * 20 years
Depreciation rate = 100%

**Step 2: Calculate the book value of the car after 20 years.**

Book value = Original cost * (1 - Depreciation rate)^Number of years
Book value = 100000$ * (1 - 100%)^20
Book value = 100000$ * 0
Book value = 0$

**Therefore, the value of the car now is $0.**

In [5]:
response = model.predict(
    """Read the Instruction below and provide an answer.

### INSTRUCTION:
In this task, you are given an input list A. You need to find all the elements of the list that are numbers and calculate their sum.

['i', 'P', 'h', '849', 'e']


### RESPONSE:""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model: 849


In [7]:
response = model.predict(
    """Read the Instruction below and provide an answer.

### INSTRUCTION:
Question:
I went to the market and bought 10 apples. I gave 2 apples to the neighbor and 2 to the repairman.
Then I went again and bought 5 more apples and I ate 1 apple.
Answer: How many apples did I remain with?

### RESPONSE:""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model: 10 - 2 - 2 = 6 apples.
6 + 5 = 11 apples.
11 - 1 = 10 apples.


In [9]:
response = model.predict(
    """Read the Instruction below and provide an answer.

### INSTRUCTION:
Question: I had a car 20 years ago and at that time I was 37. Answer: How old I am now?


### RESPONSE:""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model: 57


# Zero-Shot Prompting

In [39]:
response = model.predict(
    """Classify the text into neutral, negative or positive.
Text: I think the vacation is okay.
Sentiment:? """,
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model:  The sentiment of the text is neutral. The text does not express any strong positive or negative emotion towards the vacation. The word "okay" is a neutral term that does not imply any strong feelings either way.


# Few-Shot Prompting

In [42]:
response = model.predict(
    """A "whatpu" is a small, furry animal native to Tanzania. 
An example of a sentence that uses the word whatpu is: We were traveling in Africa and we saw these very cute whatpus.

To do a "farduddle" means to jump up and down really fast.
can you give me an example that uses the word farduddle :?""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model: The children were farduddeling around the room, laughing and having fun.


# Self-Consistency

In [23]:
response = model.predict(
    """Question: There are 15 trees in the grove. Grove workers will plant trees in the grove today. After they are done,
there will be 21 trees. How many trees did the grove workers plant today?
Answer: We start with 15 trees. Later we have 21 trees. The difference must be the number of trees they planted.
So, they must have planted 21 - 15 = 6 trees. The answer is 6.

Question: If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?
Answer: There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5.

Question: Leah had 32 chocolates and her sister had 42. If they ate 35, how many pieces do they have left in total?
Answer: Leah had 32 chocolates and Leah’s sister had 42. That means there were originally 32 + 42 = 74
chocolates. 35 have been eaten. So in total they still have 74 - 35 = 39 chocolates. The answer is 39.

Question: Jason had 20 lollipops. He gave Denny some lollipops. Now Jason has 12 lollipops. How many lollipops
did Jason give to Denny?
Answer: Jason had 20 lollipops. Since he only has 12 now, he must have given the rest to Denny. The number of
lollipops he has given to Denny must have been 20 - 12 = 8 lollipops. The answer is 8.

Question: Shawn has five toys. For Christmas, he got two toys each from his mom and dad. How many toys does
he have now?
Answer: He has 5 toys. He got 2 from mom, so after that he has 5 + 2 = 7 toys. Then he got 2 more from dad, so
in total he has 7 + 2 = 9 toys. The answer is 9.

Question: There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
Answer: There are 4 days from monday to thursday. 5 computers were added each day. That means in total 4 * 5 =
20 computers were added. There were 9 computers in the beginning, so now there are 9 + 20 = 29 computers.
The answer is 29.

Question: Michael had 58 golf balls. On tuesday, he lost 23 golf balls. On wednesday, he lost 2 more. How many
golf balls did he have at the end of wednesday?
Answer: Michael initially had 58 balls. He lost 23 on Tuesday, so after that he has 58 - 23 = 35 balls. On
Wednesday he lost 2 more so now he has 35 - 2 = 33 balls. The answer is 33.


Question: Olivia has 23 dollars, she bought 5 bagels for 3 each. 
How much money does she have left?
Answer: 

""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model: Olivia bought 5 bagels for 3 each, so she spent 5 * 3 = 15 dollars. She has 23 - 15 = 8 dollars left. The answer is 8.


# Generated Knowledge Prompting

In [43]:
response = model.predict(
    """Question: Part of golf is trying to get a higher point total than others. Yes or No?

Knowledge: The objective of golf is to play a set of holes in the least number of strokes. A round of golf typically consists of 18 holes. 
Each hole is played once in the round on a standard golf course. 
Each stroke is counted as one point, and the total number of strokes is used to determine the winner of the game.

Explain and Answer:

""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model: The answer is yes.

The knowledge states that "Each stroke is counted as one point, and the total number of strokes is used to determine the winner of the game". So part of golf is trying to get a higher point total than others.


# Hallucinations

In [44]:
response = model.predict(
    """How many 'm's are in the word 'Weather'?

""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model: There is only one 'm' in the word 'Weather'.


In [45]:
response = model.predict(
    """question: How many 'm's are in the word 'Weather'?
your previous answer: There is only one 'm' in the word 'Weather'.
question: Are you sure?
answer:?

""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model: Yes, I am sure. The word 'Weather' has only one 'm'.


In [27]:
def generate():

  model = GenerativeModel("gemini-1.0-pro-001")
  responses = model.generate_content(
    """Read the Instruction below and provide an answer.

### INSTRUCTION:
How many 'm's are in the word 'Weather'?""",
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.1,
        "top_p": 1
    },
    safety_settings={
          generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    },
    stream=True,
  )
  
  for response in responses:
    print(response.text, end="")

generate()

2

In [28]:
def generate():

  model = GenerativeModel("gemini-1.0-pro-001")
  responses = model.generate_content(
    """question: How many 'm's are in the word 'Weather'?
your previous answer: There is only one 'm' in the word 'Weather'.
question: Are you sure?
answer?""",
    generation_config={
        "max_output_tokens": 2048,
        "temperature": 0.1,
        "top_p": 1
    },
    safety_settings={
          generative_models.HarmCategory.HARM_CATEGORY_HATE_SPEECH: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
          generative_models.HarmCategory.HARM_CATEGORY_HARASSMENT: generative_models.HarmBlockThreshold.BLOCK_MEDIUM_AND_ABOVE,
    },
    stream=True,
  )
  
  for response in responses:
    print(response.text, end="")

generate()

Yes, I am sure. The word "Weather" has only one 'm'.

In [47]:
from vertexai.language_models import CodeGenerationModel

In [48]:




parameters = {
    "candidate_count": 1,
    "max_output_tokens": 1024,
    "temperature": 0.1
}
model = CodeGenerationModel.from_pretrained("code-bison")
response = model.predict(
     """Give me code in python to open an image and transform the binary in base64. You only can use python native libraries""",
    **parameters
)
print(f"Response from Model: {response.text}")

Response from Model: ```python
import base64
from PIL import Image

# Open the image
image = Image.open("image.jpg")

# Convert the image to a binary string
binary_data = image.tobytes()

# Encode the binary string in base64
base64_data = base64.b64encode(binary_data)

# Print the base64-encoded string
print(base64_data)
```
